# Imports

In [21]:
import sys
import os
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import requests as rs
import pandas as pd
from scipy.io import loadmat
from scipy.stats import zscore
from scipy.signal import spectrogram
from scipy.signal import periodogram
from scipy.signal import iirnotch
from scipy.signal import find_peaks
from scipy.signal import welch
from voltage_imaging import *
import random
import shutil

# Functions

# Loading file

In [2]:
meta = loadMeta()
dict_data = dict()
for fname in meta['File'].unique():
    path = os.path.join('..', 'data', fname.split('_')[0], fname + '.mes')
    dict_data[fname] = loadmat(path, squeeze_me = True)

# Plotting all recordings

## Selecting recordings to display

In [3]:
meta_plot = meta

## Plotting cell body and kymograph

### Raw

To render a movie, install ffmpeg and run a command similar to:
ffmpeg -r 30 -f image2 -s 1300x1000 -i frame%05d.png -vcodec libx264 -crf 20 -pix_fmt yuv420p kymograph.mp4

In [19]:
random.seed(42)
my_dpi = 300

meta_plot = meta.query('(File == "20211125_M_C_1") & (Measure_ID == 5)')

show = True
dir_images = 'kymograph_frames_V2'
os.makedirs(dir_images, exist_ok = True)

nrow = 1
ncol = 2
framerate = 30
frame_count = 0
cur_row = 0
    
#Looping through experiments
for cnt,r in meta_plot.iterrows():

    cur_mat= getRecordingData(dict_data, r)
    
    # for cur_row in range(0,cur_mat.shape[1]):
    while cur_row < cur_mat.shape[1]:
        
        if cur_row == 0:
            circle_duration = 10
        elif cur_row <= 10:
            circle_duration = 1
        else:
            circle_duration = 0.1
        
        angle_list = np.linspace(-np.pi, +np.pi, num = int(framerate*circle_duration), endpoint = False)
        if circle_duration == 0.1:
            angle_list += random.random() * np.pi/3
        
        if cur_row > 50:
            angle_list = list([random.random() * np.pi * 2])
            
        if cur_row > 100:
            cur_row += 3
            if cur_row >= cur_mat.shape[1]:
                cur_row = cur_mat.shape[1] - 1
        
        for i,angle in enumerate(angle_list):
            cursor_figure = 1
            # plt.figure(figsize = (10, 7))
            plt.figure(figsize=(1300/my_dpi, 1000/my_dpi), dpi=my_dpi)
            plt.subplot(nrow, ncol, cursor_figure)
            mat_plot = cur_mat.T
            mat_plot = np.log(mat_plot)
            mat_plot[cur_row, int(i / len(angle_list) * mat_plot.shape[1]):] = np.nan
            mat_plot[cur_row+1:,:] = np.nan
            ax = plt.imshow(mat_plot, vmin = 5, vmax = 6.5, aspect = 'auto')
            plt.ylim(cur_row + 1, -0.5,)
            plt.xticks([])
            plt.yticks([])
            plt.gca().axis('off')
            cursor_figure += 1        

            plt.subplot(nrow, ncol, cursor_figure)
            xcircle = 220
            ycircle = 272
            window = 90
            radius = 25
            overview = getRecordingData(dict_data, r, True)
            ax = plt.imshow(np.log(overview), cmap = 'Greys_r', vmax = 4.9, vmin = 4.75)
            plt.gca().axis('off')
            circle = plt.Circle((220, 272), radius, color='r', fill = None, linewidth = 3)
            plt.scatter(xcircle + radius*np.cos(angle), ycircle + radius*np.sin(angle), c = 'cyan', zorder = 100, s = 100)
            plt.gca().add_patch(circle)
            plt.xlim(xcircle - window, xcircle + window)
            plt.ylim(ycircle - window, ycircle + window)
            
            time = 2*cur_row + (i/len(angle_list)) * 2
            if cur_row == cur_mat.shape[1] - 1:
                time = 3000
                
            unit = 'ms'
            if time < 4:
                time *= 1000
                unit = 'µs'
                
            plt.suptitle('Time = ' + str(int(time)) + ' ' + unit, fontsize = 20)
            plt.savefig(dir_images + '/frame' + format(frame_count, '05d'))
            plt.close()
            frame_count += 1
            
        cur_row += 1

In [24]:
last_frame = dir_images + '/frame' + format(frame_count-1, '05d') + '.png'
duration_last_image = 5
n_frames = duration_last_image * framerate
for cur_frame in range(frame_count, frame_count+1+n_frames):
    new_frame = dir_images + '/frame' + format(cur_frame, '05d') + '.png'
    shutil.copyfile(last_frame, new_frame)